#Imports


In [0]:
import pandas as pd
import numpy as np
import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Convolution2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Dense, Flatten, AveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
from google.colab import drive

Using TensorFlow backend.


#Getting dataset from my drive

In [0]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
cd gdrive

/content/gdrive


In [0]:
cd My Drive

/content/gdrive/My Drive


In [0]:
ls

 10308296_1850492438579598_470283656400563920_n.jpg
 1515074182688366687413.jpg
'1515074182688366687413 - rohit nagotkar.jpg'
 52138
 Abstract_Code_Diggers/
 asdf.pdf
 athena.png
'Attendance CA.gsheet'
 Attendance.gsheet
 Books/
 bot.gdoc
 builders.rohit.gsheet
 call.docx
'Class notes.gdoc'
 Classroom/
'Colab Notebooks'/
'Colleges and schools.gsheet'
'Contacts from whatsapp.xlsx'
'Copy of Marina · SlidesCarnival.gslides'
 cretronix2/
 data.csv
 Debate.gdoc
'Delegate Agreement.pdf'
'Document from Rohit'
 drawings/
'Electric nose.docx'
'Electric nose.gdoc'
 EMAIL-IDs.gsheet
 exp_1.m
 fer2013.csv
 fill.xlsx
'Google Photos'/
'How to get started with Drive.pdf'
 IMG-20170913-WA0005.jpg
'Imogen · SlidesCarnival.gslides'
 interview.xlsx
 KRIEG.gdoc
'Lalit GA-1.JPG'
'Lce .gdoc'
'LIQUID CRYSTALLINE ELASTOMERS.gdoc'
'Manipal Hackathon.gdoc'
'mech, civil, electrical and biotech.gslides'
 Model.01-0.2053.hdf5
 Model.01-0.2513.hdf5
 Model.01-0.2547.hdf5
 Model.01-0.3380.hdf5
 Model.02-0.2602.hdf5
 

In [0]:
batch_size = 32
num_epochs = 25

#obtain pixel values from dataset

In [0]:
def pandas_vector_to_list(pandas_df):
    py_list = [item[0] for item in pandas_df.values.tolist()]
    return py_list

def process_pixels(pixels, img_size=48):
    pixels_as_list = pandas_vector_to_list(pixels)
    np_image_array = []
    for index, item in enumerate(pixels_as_list):
        data = np.zeros((img_size, img_size), dtype=np.uint8)
        pixel_data = item.split()
        for i in range(0, img_size):
            pixel_index = i * img_size
            data[i] = pixel_data[pixel_index:pixel_index + img_size]
        np_image_array.append(np.array(data))
    np_image_array = np.array(np_image_array)
    np_image_array = np_image_array.astype('float32') / 255.0
    return np_image_array



#Read dataset

In [0]:
width = height = 48
depth = 1
data = pd.read_csv('fer2013.csv')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
pixels     35887 non-null object
Usage      35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


#Split into train test and validation

In [0]:
train_set = data[(data.Usage == 'Training')]
validation_set = data[(data.Usage == 'PublicTest')]
test_set = data[(data.Usage == 'PrivateTest')]

print('Samples per emotion:')
print(data.emotion.value_counts())


Samples per emotion:
3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: emotion, dtype: int64


#final input to neural net

In [0]:
X_train = process_pixels(train_set[['pixels']])
X_test = process_pixels(test_set[['pixels']])
X_validation = process_pixels(validation_set[['pixels']])
print(test_set[10:11])
num_train = X_train.shape[0]
num_validation = X_validation.shape[0]
num_test = X_test.shape[0]

X_train = X_train.reshape(num_train, width, height, depth)
X_validation = X_validation.reshape(num_test, width, height, depth)
X_test = X_test.reshape(num_test, width, height, depth)

print('Training: ',X_train.shape)
print('Validation: ',X_validation.shape)
print('Test: ',X_test.shape)


       emotion                                             pixels        Usage
32308        5  255 255 255 252 255 212 122 107 137 178 196 21...  PrivateTest
Training:  (28709, 48, 48, 1)
Validation:  (3589, 48, 48, 1)
Test:  (3589, 48, 48, 1)


In [0]:
y_train = to_categorical(train_set.emotion, 7)
y_validation = to_categorical(validation_set.emotion, 7)
y_test = to_categorical(test_set.emotion, 7)
print('Training: ',y_train.shape)
print('Validation: ',y_validation.shape)
print('Test: ',y_test.shape)

Training:  (28709, 7)
Validation:  (3589, 7)
Test:  (3589, 7)


#Model

In [0]:
model = Sequential()
#1
model.add(Convolution2D(filters = 64,kernel_size= 3, activation = 'relu', padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.3))
#2
model.add(Convolution2D(filters = 128,kernel_size= 3,activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.3))
#3
model.add(Convolution2D(filters = 128,kernel_size= 3,activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.3))
#4
model.add(Convolution2D(filters = 256,kernel_size = 3, padding='same',activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.3))
#5
model.add(Convolution2D(filters = 256,kernel_size = 3, padding='same',activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.3))
#6
model.add(Convolution2D(filters = 512, kernel_size = 3,activation = 'relu', padding='same'))
model.add(Convolution2D(512, (1, 3), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.3))
#7
model.add(Convolution2D(filters = 512, kernel_size = 3,activation = 'relu', padding='same'))
model.add(Convolution2D(512, (1, 3), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.3))
#8
model.add(Flatten())
#9
model.add(Dense(7))
model.add(Activation('softmax'))
print(model.summary())








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 128)       512      

#image augmentation

In [0]:
datagen = ImageDataGenerator(featurewise_center=False,samplewise_center=False,featurewise_std_normalization=False,
                             samplewise_std_normalization=False,zca_whitening=False,rotation_range=0,
                             width_shift_range=0.0,height_shift_range=0.0,horizontal_flip=True,
                             vertical_flip=False,)

datagen.fit(X_train)
datagen.fit(X_validation)
model.compile(loss='categorical_crossentropy',optimizer= 'adam' ,metrics=['acc'])

In [0]:
train_flow = datagen.flow(X_train, y_train, batch_size=batch_size)
validation_flow = datagen.flow(X_validation, y_validation)


#callbacks

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

filepath='Model.{epoch:02d}-{val_acc:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')


#Training the network

In [0]:
history = model.fit_generator(train_flow,steps_per_epoch=(len(X_train) / batch_size),epochs=num_epochs,verbose=1,validation_data=validation_flow,validation_steps=len(X_validation) / batch_size,callbacks=[checkpointer, reduce_lr, checkpointer])
print("break")
score = model.evaluate(X_test, y_test, steps=int(len(X_test) / batch_size))
print('test loss: ', score[0])
print('Test accuracy: ', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/25
898/897 [==============================] - 60s 67ms/step - loss: 1.9991 - acc: 0.2413 - val_loss: 1.6709 - val_acc: 0.3274

Epoch 00001: saving model to Model.01-0.3274.hdf5

Epoch 00001: saving model to Model.01-0.3274.hdf5
Epoch 2/25
898/897 [==============================] - 57s 63ms/step - loss: 1.6280 - acc: 0.3819 - val_loss: 1.4600 - val_acc: 0.4452

Epoch 00002: saving model to Model.02-0.4452.hdf5

Epoch 00002: saving model to Model.02-0.4452.hdf5
Epoch 3/25
898/897 [==============================] - 57s 63ms/step - loss: 1.4455 - acc: 0.4526 - val_loss: 1.3705 - val_acc: 0.4837

Epoch 00003: saving model to Model.03-0.4837.hdf5

Epoch 00003: saving model to Model.03-0.4837.hdf5
Epoch 4/25
898/897 [==============================] - 56s 63ms/step - loss: 1.3607 - acc: 0.4865 - val_loss: 1.3844 - val_acc: 0.4617

Epoch 00004: saving model to Model.04-0.4617.hdf5

Epoch 00004:

In [0]:
score2 = model.evaluate(X_train, y_train,verbose = 0)
print('train loss: ', score2[0])
print('train accuracy: ', score2[1])

test loss:  0.7742659898942663
Test accuracy:  0.7103695705200322


#print test values and save the model

In [0]:
y_pred = model.predict_classes(X_test)
y_true = np.asarray([np.argmax(i) for i in y_test])
temp = (y_pred-y_true)
print(np.asarray(temp).tolist())
model_json = model.to_json()
with open("model.json","w") as json_file:
     json_file.write(model_json)
np.savetxt("Y-pred.csv",y_pred,delimiter= ',')
np.savetxt("Y-true.csv",y_true,delimiter= ',')

model.save('final_weights.h5')

[6, -1, -4, 0, 2, 0, 0, 0, 0, 0, -2, 0, 2, 0, 4, 0, -2, 0, 0, -2, 0, -2, 0, 0, 0, -2, 0, 0, 0, 1, 0, -2, 0, 2, 0, 0, -3, 6, 2, 0, -2, 2, 0, 0, -2, 0, 0, 0, 3, 0, 0, 0, 0, -2, 0, 1, 0, 0, 0, 2, 0, 1, 0, 3, 3, 0, 0, 0, 0, 4, 2, -5, -1, 0, 0, 6, 0, 2, 0, 0, 0, 1, 0, -2, 3, 0, 0, 0, 0, 0, 0, 3, -4, 0, 0, 0, 2, 0, -4, -4, 0, 0, 0, 0, 0, 2, -3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, -2, 0, 2, 6, 0, 6, 0, 0, 0, 0, 2, 0, 0, 0, 4, 0, -2, 0, 2, -2, 0, 0, 0, -3, -6, 0, 5, 4, 0, 0, 0, -3, 0, -2, -2, 4, 4, 5, -3, 0, 0, 0, 0, 0, 2, 0, -2, 2, -1, -2, 0, 0, 0, 2, 4, 0, 0, -2, 0, 4, 0, 3, 0, 0, 0, 3, 0, 4, 0, 0, 4, 0, 2, -2, 0, -6, 1, 3, 4, 2, 0, 0, 0, 4, 0, -3, -2, 3, 0, 0, 2, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, -2, 0, 0, 2, 2, 0, 0, 0, -6, 0, -4, 0, 3, 0, 1, 0, -4, -1, 0, -2, 6, 6, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, -2, 0, 0, 4, -2, 0, 0, 2, 0, 0, 1, 0, 6, 0, 0, 0, 0, -4, 0, 2, 0, 0, -2, -2, 0, 0, 0, 0, 2, -4, 0, 0, 0, 0, 0, -3, 0, 0, 0, 2, 4, 0, -2, -2, 0, 0, -2, 0, 0, -4, 0

#load model

In [0]:
from keras.models import load_model
filepath = "final_weights.h5"
mod = load_model(
    filepath,
    custom_objects=None,
    compile=True
)

In [0]:
mod.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 128)       512       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 128)      